In [1]:
import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

2.4.0
2.4.3


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [3]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import time

pd.set_option('display.max_columns', 70)

start = time.time()

In [4]:
def scale_data(data):
    list_of_features = []
    list_of_outputs = []
    for column in data.columns:
        list_of_features.append(data[column])
        if column in ['Open', 'Close', 'Highest', 'Lowest', 'Volume', 'Otwarcie', 'Najwyzszy', 'Najnizszy', 'Zamkniecie', 'Wolumen']:
            list_of_outputs.append(data[column])
        
    dataset = np.transpose(list_of_features)
    output_dataset = np.transpose(list_of_outputs)
    X_scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = X_scaler.fit_transform(dataset)
    Y_scaler = MinMaxScaler(feature_range=(0,1))
    Y_scaler.fit_transform(output_dataset)

    return scaled_data, X_scaler, Y_scaler

In [5]:
def prepare_input_and_output(data, number_of_sessions=60):
    # number_of_sessions - number of considered previous sessions as an input
    X = []
    Y = []
    for i in range(number_of_sessions, data.shape[0]):
        X.append(data[i-number_of_sessions:i, :])
        Y.append(data[i, :5])
    return X, Y

In [6]:
fundamental_data = pd.read_csv('Testing_data/WiG20_full_data.csv')

fundamental_data.head()

,Spółka,Data,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen,Kwartały,Wartość księgowa,Cena/WK,Zysk,Cena/Zysk,Przychód,Cena/Przychód,Zysk operacyjny,Cena/Zysk operacyjny,Wartość księgowa Grahama,Cena/Wartość księgowa Grahama,Wartość przedsiębiorstwa,Wartość przedsiębiorstwa/Przychody,Wartość przedsiębiorstwa/EBIT,Wartość przedsiębiorstwa/EBITDA,ROE,ROA,Marża zysku operacyjnego,Marża zysku netto,Marża zysku ze sprzedaży,Marża zysku brutto,Marża zysku brutto ze sprzedaży,Rentowność operacyjna aktywów,Udział zysku netto w przepływach operacyjnych,Wskaźnik źródeł finansowania inwestycji,Zadłużenie ogólne,Zadłużenie kapitału własnego,Zadłużenie długoterminowe,Zadłużenie środków trwałych,Pokrycie aktywów trwałych kapitałami stałymi,Trwałość struktury finansowania,Zastosowanie kapitału obcego,Wskaźnik ogólnej sytuacji finansowej,Zadłużenie netto,Zadłużenie netto / EBITDA,Zadłużenie finansowe netto,Zadłużenie finansowe netto / EBITDA,I stopień pokrycia,II stopień pokrycia,Płynność gotówkowa,Płynność szybka,Płynność bieżąca,Płynność podwyższona,Pokrycie zobowiązań należnościami,Udział kapitału pracującego w aktywach,Pokrycie kosztów kapitałem obrotowym,Rotacja należności,Cykl należności,Cykl zobowiązań,Rotacja zapasów,Cykl zapasów,Rotacja majątku obrotowego,Rotacja majątku trwałego,Rotacja majątku ogółem,Cykl operacyjny,Cykl konwersji gotówki
0,ACP,2005-02-28,16.358,16.854,16.358,16.526,492612.0,2004/Q4,6.55,2.50,0.02,817.90,23.01,0.71,1.03,15.88,-4.77,-3.43,33.99,1.48,21.97,33.1,0.32,0.08,4.46,0.09,5.43,-1.11,19.58,4.06,-1.68,108.82,0.73,2.84,0.63,0.34,0.93,0.42,1.34,0.77,292874000.0,9.04,127506000.0,3.94,0.57,0.93,0.16,0.89,0.96,0.32,0.55,-0.02,-0.04,2.89,126.0,229.0,19.74,18.0,1.66,2.0,0.91,144.0,-85.0
1,ACP,2005-03-01,16.741,16.741,15.979,16.308,293232.0,2004/Q4,6.55,2.56,0.02,837.05,23.01,0.73,1.03,16.25,-4.77,-3.51,33.99,1.48,21.97,33.1,0.32,0.08,4.46,0.09,5.43,-1.11,19.58,4.06,-1.68,108.82,0.73,2.84,0.63,0.34,0.93,0.42,1.34,0.77,292874000.0,9.04,127506000.0,3.94,0.57,0.93,0.16,0.89,0.96,0.32,0.55,-0.02,-0.04,2.89,126.0,229.0,19.74,18.0,1.66,2.0,0.91,144.0,-85.0
2,ACP,2005-03-02,16.308,16.414,15.594,15.817,163743.0,2004/Q4,6.55,2.49,0.02,815.40,23.01,0.71,1.03,15.83,-4.77,-3.42,33.99,1.48,21.97,33.1,0.32,0.08,4.46,0.09,5.43,-1.11,19.58,4.06,-1.68,108.82,0.73,2.84,0.63,0.34,0.93,0.42,1.34,0.77,292874000.0,9.04,127506000.0,3.94,0.57,0.93,0.16,0.89,0.96,0.32,0.55,-0.02,-0.04,2.89,126.0,229.0,19.74,18.0,1.66,2.0,0.91,144.0,-85.0
3,ACP,2005-03-03,15.817,16.035,15.650,15.650,116137.0,2004/Q4,6.55,2.41,0.02,790.85,23.01,0.69,1.03,15.36,-4.77,-3.32,33.99,1.48,21.97,33.1,0.32,0.08,4.46,0.09,5.43,-1.11,19.58,4.06,-1.68,108.82,0.73,2.84,0.63,0.34,0.93,0.42,1.34,0.77,292874000.0,9.04,127506000.0,3.94,0.57,0.93,0.16,0.89,0.96,0.32,0.55,-0.02,-0.04,2.89,126.0,229.0,19.74,18.0,1.66,2.0,0.91,144.0,-85.0
4,ACP,2005-03-04,15.707,15.923,15.650,15.923,48766.0,2004/Q4,6.55,2.40,0.02,785.35,23.01,0.68,1.03,15.25,-4.77,-3.29,33.99,1.48,21.97,33.1,0.32,0.08,4.46,0.09,5.43,-1.11,19.58,4.06,-1.68,108.82,0.73,2.84,0.63,0.34,0.93,0.42,1.34,0.77,292874000.0,9.04,127506000.0,3.94,0.57,0.93,0.16,0.89,0.96,0.32,0.55,-0.02,-0.04,2.89,126.0,229.0,19.74,18.0,1.66,2.0,0.91,144.0,-85.0


In [7]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [8]:
# Firstly we learn based on prices and volume only as a reference point for further predictions
X_all_ref = []
Y_all_ref = []
X_scalers_ref = []
Y_scalers_ref = []
results = []

reference_data = fundamental_data[['Spółka', 'Otwarcie', 'Najwyzszy', 'Najnizszy', 'Zamkniecie', 'Wolumen']].dropna()

for stock in list(reference_data['Spółka'].unique()):
    scaled_stock, X_scaler, Y_scaler = scale_data(reference_data.loc[reference_data['Spółka'] == stock].drop(columns=['Spółka']))
    X_scalers_ref.append(X_scaler)
    Y_scalers_ref.append(Y_scaler)
    X_single, Y_single = prepare_input_and_output(scaled_stock)
    X_all_ref = X_all_ref + X_single
    Y_all_ref = Y_all_ref + Y_single
X_all_ref, Y_all_ref = np.array(X_all_ref), np.array(Y_all_ref)
X_train_ref, X_valid_ref, Y_train_ref, Y_valid_ref = train_test_split(X_all_ref, Y_all_ref, test_size=0.2, shuffle=False)

print(X_train_ref.shape)
print(X_valid_ref.shape)
print(Y_train_ref.shape)
print(Y_valid_ref.shape)

loss_results_ref = []
val_loss_results_ref = []
for i in range(3):
    model = Sequential()
    model.add(LSTM(32, input_shape=[X_train_ref.shape[1], X_train_ref.shape[2]]))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(Y_train_ref.shape[1]))
    model.compile(loss='mse', optimizer='adam')
    history = model.fit(X_train_ref, Y_train_ref, epochs=7, validation_data=(X_valid_ref, Y_valid_ref))
    loss_results_ref.append(history.history['loss'][-1])
    val_loss_results_ref.append(history.history['val_loss'][-1])
results.append({'indicator': 'reference', 'loss': np.mean(loss_results_ref), 'val_loss': np.mean(val_loss_results_ref)})
results = pd.DataFrame(results)

(48433, 60, 5)
(12109, 60, 5)
(48433, 5)
(12109, 5)
Epoch 1/7
1514/1514 [==============================] - 9s 5ms/step - loss: 0.0147 - val_loss: 0.0011
Epoch 2/7
1514/1514 [==============================] - 6s 4ms/step - loss: 6.5356e-04 - val_loss: 9.5484e-04
Epoch 3/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.9408e-04 - val_loss: 9.1634e-04
Epoch 4/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.6181e-04 - val_loss: 8.7726e-04
Epoch 5/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.1948e-04 - val_loss: 8.8412e-04
Epoch 6/7
1514/1514 [==============================] - 6s 4ms/step - loss: 4.9637e-04 - val_loss: 8.2433e-04
Epoch 7/7
1514/1514 [==============================] - 6s 4ms/step - loss: 4.6903e-04 - val_loss: 8.2544e-04
Epoch 1/7
1514/1514 [==============================] - 7s 4ms/step - loss: 0.0299 - val_loss: 0.0010
Epoch 2/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.6802e-04 - val_

In [9]:

# iterate through columns as only one at a time is added to the reference data
for column in fundamental_data.columns[8:]:
    X_all = []
    Y_all = []
    X_scalers = []
    Y_scalers = []
    print(column)
    data = fundamental_data[['Spółka', 'Otwarcie', 'Najwyzszy', 'Najnizszy', 'Zamkniecie', 'Wolumen', column]].dropna()
    
    # preparation of input X and output Y sets needs to be done on stocks seperately
    for stock in list(data['Spółka'].unique()):
        scaled_stock, X_scaler, Y_scaler = scale_data(data.loc[data['Spółka'] == stock].drop(columns=['Spółka']))
        X_scalers.append(X_scaler)
        Y_scalers.append(Y_scaler)
    
        X_single, Y_single = prepare_input_and_output(scaled_stock)
        X_all = X_all + X_single
        Y_all = Y_all + Y_single
    X_all, Y_all = np.array(X_all), np.array(Y_all)
    X_train, X_valid, Y_train, Y_valid = train_test_split(X_all, Y_all, test_size=0.2, shuffle=False)
#     print('Shape of training input matrix: ' + str(X_train.shape))
#     print('Shape of training output matrix: ' + str(Y_train.shape))
#     print('Shape of validation input matrix: ' + str(X_valid.shape))
#     print('Shape of validation output matrix: ' + str(Y_valid.shape))
    
    # neural network
    loss_results = []
    val_loss_results = []
    for i in range(3):
        model = Sequential()
        model.add(LSTM(32, input_shape=[X_train.shape[1], X_train.shape[2]]))
        model.add(Dense(10, activation='relu'))
        model.add(Dense(Y_train.shape[1]))
        model.compile(loss='mse', optimizer='adam')
        history = model.fit(X_train, Y_train, epochs=7, validation_data=(X_valid, Y_valid))
        loss_results.append(history.history['loss'][-1])
        val_loss_results.append(history.history['val_loss'][-1])
    results.append({'indicator': column, 'loss': np.mean(loss_results), 'val_loss': np.mean(val_loss_results)}, ignore_index=True)

    

Wartość księgowa
Epoch 1/7
1514/1514 [==============================] - 7s 4ms/step - loss: 0.0137 - val_loss: 0.0011
Epoch 2/7
1514/1514 [==============================] - 6s 4ms/step - loss: 6.0900e-04 - val_loss: 9.3414e-04
Epoch 3/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.5698e-04 - val_loss: 9.0627e-04
Epoch 4/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.5557e-04 - val_loss: 0.0010
Epoch 5/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.5444e-04 - val_loss: 8.9368e-04
Epoch 6/7
1514/1514 [==============================] - 6s 4ms/step - loss: 4.9043e-04 - val_loss: 7.9969e-04
Epoch 7/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.2424e-04 - val_loss: 8.4077e-04
Epoch 1/7
1514/1514 [==============================] - 7s 4ms/step - loss: 0.0154 - val_loss: 0.0011
Epoch 2/7
1514/1514 [==============================] - 6s 4ms/step - loss: 6.3700e-04 - val_loss: 0.0010
Epoch 3/7
1514/1514 [=====

Epoch 7/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.1868e-04 - val_loss: 8.1616e-04
Epoch 1/7
1514/1514 [==============================] - 7s 4ms/step - loss: 0.0089 - val_loss: 0.0012
Epoch 2/7
1514/1514 [==============================] - 6s 4ms/step - loss: 6.4871e-04 - val_loss: 0.0010
Epoch 3/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.6760e-04 - val_loss: 9.4796e-04
Epoch 4/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.2086e-04 - val_loss: 8.5067e-04
Epoch 5/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.1137e-04 - val_loss: 8.2030e-04
Epoch 6/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.1417e-04 - val_loss: 8.2746e-04
Epoch 7/7
1514/1514 [==============================] - 6s 4ms/step - loss: 4.7540e-04 - val_loss: 8.4984e-04
Przychód
Epoch 1/7
1514/1514 [==============================] - 7s 4ms/step - loss: 0.0077 - val_loss: 0.0012
Epoch 2/7
1514/1514 [=========

1514/1514 [==============================] - 6s 4ms/step - loss: 5.0786e-04 - val_loss: 8.1039e-04
Epoch 6/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.0235e-04 - val_loss: 8.4262e-04
Epoch 7/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.1318e-04 - val_loss: 8.4476e-04
Epoch 1/7
1514/1514 [==============================] - 7s 4ms/step - loss: 0.0324 - val_loss: 0.0012
Epoch 2/7
1514/1514 [==============================] - 6s 4ms/step - loss: 6.6452e-04 - val_loss: 0.0011
Epoch 3/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.7703e-04 - val_loss: 0.0011
Epoch 4/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.2962e-04 - val_loss: 9.1445e-04
Epoch 5/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.2704e-04 - val_loss: 9.9338e-04
Epoch 6/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.0088e-04 - val_loss: 8.9228e-04
Epoch 7/7
1514/1514 [========================

1212/1212 [==============================] - 5s 4ms/step - loss: 5.6926e-04 - val_loss: 5.7969e-04
Epoch 4/7
1212/1212 [==============================] - 5s 4ms/step - loss: 5.7795e-04 - val_loss: 5.4883e-04
Epoch 5/7
1212/1212 [==============================] - 5s 4ms/step - loss: 5.1917e-04 - val_loss: 5.3098e-04
Epoch 6/7
1212/1212 [==============================] - 5s 4ms/step - loss: 4.8977e-04 - val_loss: 5.2376e-04
Epoch 7/7
1212/1212 [==============================] - 5s 4ms/step - loss: 5.2472e-04 - val_loss: 5.0124e-04
Wartość przedsiębiorstwa/Przychody
Epoch 1/7
1212/1212 [==============================] - 6s 4ms/step - loss: 0.0092 - val_loss: 7.4030e-04
Epoch 2/7
1212/1212 [==============================] - 5s 4ms/step - loss: 6.4456e-04 - val_loss: 6.5722e-04
Epoch 3/7
1212/1212 [==============================] - 5s 4ms/step - loss: 5.8685e-04 - val_loss: 5.8500e-04
Epoch 4/7
1212/1212 [==============================] - 5s 4ms/step - loss: 6.4142e-04 - val_loss: 5.3184e-0

1514/1514 [==============================] - 6s 4ms/step - loss: 4.8776e-04 - val_loss: 8.5433e-04
Epoch 1/7
1514/1514 [==============================] - 7s 4ms/step - loss: 0.0138 - val_loss: 0.0011
Epoch 2/7
1514/1514 [==============================] - 6s 4ms/step - loss: 6.5557e-04 - val_loss: 0.0012
Epoch 3/7
1514/1514 [==============================] - 6s 4ms/step - loss: 6.0926e-04 - val_loss: 9.0199e-04
Epoch 4/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.8369e-04 - val_loss: 8.7784e-04
Epoch 5/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.1490e-04 - val_loss: 8.5836e-04
Epoch 6/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.2136e-04 - val_loss: 8.1947e-04
Epoch 7/7
1514/1514 [==============================] - 6s 4ms/step - loss: 4.7228e-04 - val_loss: 8.3141e-04
Epoch 1/7
1514/1514 [==============================] - 8s 4ms/step - loss: 0.0079 - val_loss: 0.0011
Epoch 2/7
1514/1514 [============================

...

Epoch 6/7
1212/1212 [==============================] - 5s 4ms/step - loss: 4.9087e-04 - val_loss: 5.2072e-04
Epoch 7/7
1212/1212 [==============================] - 5s 4ms/step - loss: 4.6505e-04 - val_loss: 4.9860e-04
Epoch 1/7
1212/1212 [==============================] - 6s 4ms/step - loss: 0.0214 - val_loss: 8.0115e-04
Epoch 2/7
1212/1212 [==============================] - 5s 4ms/step - loss: 6.6255e-04 - val_loss: 6.8573e-04
Epoch 3/7
1212/1212 [==============================] - 5s 4ms/step - loss: 6.0560e-04 - val_loss: 5.9243e-04
Epoch 4/7
1212/1212 [==============================] - 5s 4ms/step - loss: 5.8878e-04 - val_loss: 5.7829e-04
Epoch 5/7
1212/1212 [==============================] - 5s 4ms/step - loss: 5.4429e-04 - val_loss: 5.5391e-04
Epoch 6/7
1212/1212 [==============================] - 5s 4ms/step - loss: 5.1973e-04 - val_loss: 5.3538e-04
Epoch 7/7
1212/1212 [==============================] - 5s 4ms/step - loss: 5.1166e-04 - val_loss: 5.2559e-04
Epoch 1/7
1212/1212 [==

Epoch 4/7
1212/1212 [==============================] - 5s 4ms/step - loss: 5.9763e-04 - val_loss: 5.9877e-04
Epoch 5/7
1212/1212 [==============================] - 5s 4ms/step - loss: 5.4282e-04 - val_loss: 5.3117e-04
Epoch 6/7
1212/1212 [==============================] - 5s 4ms/step - loss: 4.9180e-04 - val_loss: 5.2284e-04
Epoch 7/7
1212/1212 [==============================] - 5s 4ms/step - loss: 4.9944e-04 - val_loss: 5.1762e-04
Udział kapitału pracującego w aktywach
Epoch 1/7
1212/1212 [==============================] - 6s 4ms/step - loss: 0.0219 - val_loss: 7.6965e-04
Epoch 2/7
1212/1212 [==============================] - 5s 4ms/step - loss: 7.0509e-04 - val_loss: 6.3730e-04
Epoch 3/7
1212/1212 [==============================] - 5s 4ms/step - loss: 5.7830e-04 - val_loss: 5.9434e-04
Epoch 4/7
1212/1212 [==============================] - 5s 4ms/step - loss: 5.8402e-04 - val_loss: 5.7824e-04
Epoch 5/7
1212/1212 [==============================] - 5s 4ms/step - loss: 5.5131e-04 - val_l

Epoch 2/7
1514/1514 [==============================] - 6s 4ms/step - loss: 6.6023e-04 - val_loss: 0.0010
Epoch 3/7
1514/1514 [==============================] - 6s 4ms/step - loss: 6.1865e-04 - val_loss: 8.8627e-04
Epoch 4/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.4796e-04 - val_loss: 8.3041e-04
Epoch 5/7
1514/1514 [==============================] - 6s 4ms/step - loss: 4.7826e-04 - val_loss: 8.4828e-04
Epoch 6/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.1186e-04 - val_loss: 8.1970e-04
Epoch 7/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.0419e-04 - val_loss: 8.1732e-04
Epoch 1/7
1514/1514 [==============================] - 8s 4ms/step - loss: 0.0270 - val_loss: 0.0010
Epoch 2/7
1514/1514 [==============================] - 6s 4ms/step - loss: 6.1221e-04 - val_loss: 8.9548e-04
Epoch 3/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.4852e-04 - val_loss: 9.4513e-04
Epoch 4/7
1514/1514 [==========

1146/1146 [==============================] - 5s 5ms/step - loss: 5.3924e-04 - val_loss: 4.6702e-04
Rotacja majątku obrotowego
Epoch 1/7
1212/1212 [==============================] - 8s 5ms/step - loss: 0.0111 - val_loss: 8.3649e-04
Epoch 2/7
1212/1212 [==============================] - 6s 5ms/step - loss: 7.3930e-04 - val_loss: 7.1405e-04
Epoch 3/7
1212/1212 [==============================] - 6s 5ms/step - loss: 6.2519e-04 - val_loss: 5.8909e-04
Epoch 4/7
1212/1212 [==============================] - 6s 5ms/step - loss: 5.9560e-04 - val_loss: 5.6589e-04
Epoch 5/7
1212/1212 [==============================] - 5s 4ms/step - loss: 5.2851e-04 - val_loss: 5.3327e-04
Epoch 6/7
1212/1212 [==============================] - 5s 4ms/step - loss: 5.0455e-04 - val_loss: 5.3977e-04
Epoch 7/7
1212/1212 [==============================] - 6s 5ms/step - loss: 4.7981e-04 - val_loss: 5.0682e-04
Epoch 1/7
1212/1212 [==============================] - 7s 4ms/step - loss: 0.0093 - val_loss: 7.4309e-04
Epoch 2/7


Epoch 5/7
1146/1146 [==============================] - 5s 4ms/step - loss: 5.4147e-04 - val_loss: 4.6719e-04
Epoch 6/7
1146/1146 [==============================] - 5s 4ms/step - loss: 5.2145e-04 - val_loss: 4.4896e-04
Epoch 7/7
1146/1146 [==============================] - 5s 4ms/step - loss: 5.0826e-04 - val_loss: 4.4654e-04
Epoch 1/7
1146/1146 [==============================] - 6s 5ms/step - loss: 0.0156 - val_loss: 6.6001e-04
Epoch 2/7
1146/1146 [==============================] - 5s 4ms/step - loss: 6.6327e-04 - val_loss: 5.4596e-04
Epoch 3/7
1146/1146 [==============================] - 5s 4ms/step - loss: 5.6461e-04 - val_loss: 5.1845e-04
Epoch 4/7
1146/1146 [==============================] - 5s 4ms/step - loss: 5.6811e-04 - val_loss: 4.9080e-04
Epoch 5/7
1146/1146 [==============================] - 5s 4ms/step - loss: 5.2691e-04 - val_loss: 4.7569e-04
Epoch 6/7
1146/1146 [==============================] - 5s 4ms/step - loss: 5.0435e-04 - val_loss: 4.6972e-04
Epoch 7/7
1146/1146 [==

In [10]:
results

,indicator,loss,val_loss
0,reference,0.000498,0.000838


In [11]:
end = time.time()
print('Time of executing the script: ' + str(round((end - start)/3600, 2)) + 'h')

Time of executing the script: 1.82h
